In [1]:
from sknn.mlp import Regressor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
import matplotlib
import sklearn
from sknn.mlp import Regressor, Layer
from sklearn_pandas import DataFrameMapper
matplotlib.style.use('ggplot')
%matplotlib inline

/home/nikhil/.conda/envs/sarcoma/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('india_surroundings_1mar2016 _28_feb_2017.csv')
del df['IRIS ID'], df['Year'], df['Month'], df['Day'], df['Time UTC'], df['Region']
df = df.iloc[::-1]
#df = pd.get_dummies(df)

In [3]:
train_test_ratio = .7
train_size = int(df.shape[0] * train_test_ratio)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]
df.head()

     Mag    Lat    Lon  Depth km   Timestamp
279  4.6  29.75  69.93      10.0  1456900511
278  4.0  35.53  70.24      89.5  1457132148
277  4.2  36.49  71.10     101.3  1457232360
276  4.6  20.98  95.02     101.0  1457320438
275  4.5  22.76  93.94      50.0  1457353381

In [4]:
mapper = DataFrameMapper([
       # (['Region'], None),
        (['Mag'],sklearn.preprocessing.StandardScaler()),
        (['Depth km'],sklearn.preprocessing.StandardScaler()),
        (['Lat'],sklearn.preprocessing.StandardScaler()),
        (['Lon'],sklearn.preprocessing.StandardScaler()),
        (['Timestamp'], sklearn.preprocessing.StandardScaler()) 
    ],default = None)
train_data = mapper.fit_transform(train_data)
#train_data[['Mag','Lat','Lon','Timestamp','Depth km']] = train_data[['Mag','Lat','Lon','Timestamp','Depth km']].astype(float)
test_data = mapper.transform(test_data)
#test_data[['Mag','Lat','Lon','Timestamp','Depth km']] = test_data[['Mag','Lat','Lon','Timestamp','Depth km']].astype(float)


/home/nikhil/.conda/envs/sarcoma/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [5]:
past = 14  # number of past earthquakes to consider
def getXandY(data):
    X, Y = [], []
    for i in range(data.shape[0]-past):
        X += [data[i:i+past].flatten()]
        Y += [data[i+past]]
    return X, Y
train_X, train_Y = getXandY(train_data)
test_X, test_Y = getXandY(test_data)
train_X, train_Y, test_X, test_Y = np.array(train_X), np.array(train_Y), np.array(test_X), np.array(test_Y)
print(np.shape(train_X), np.shape(train_Y))
print(np.shape(test_X), np.shape(test_Y))

(182, 70) (182, 5)
(70, 70) (70, 5)


In [6]:
errors = []
def store_stats(avg_valid_error, avg_train_error, **_):
    errors.append((avg_valid_error, avg_train_error))

nn = Regressor(
    layers=[
         Layer("Linear",units= 1),
        Layer("Sigmoid",units = 20),
        Layer("Linear")],
#    callback={'on_epoch_finish': store_stats},
    learning_rate=0.01,
    n_iter= 1000)

In [13]:
gs = GridSearchCV(nn, param_grid={
    'learning_rate': [0.01, 0.001],
    'hidden0__units': [12, 40, 100],
    'hidden0__type': ["Rectifier", "Sigmoid", "Tanh"],
    #'hidden1__units': [12, 40, 100],
    #'hidden1__type': ["Rectifier", "Sigmoid", "Tanh"],
})
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    gs.fit(train_X, train_Y)
print(gs.best_params_)

KeyboardInterrupt: 

ImportError: cannot import name 'downsample'

NameError: name 'lasagne' is not defined